In [2]:
import numpy as np
import pandas as pd

import sklearn

from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit import Chem
from rdkit.Chem import DataStructs
from numpy import linalg
from pickle import load
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import os

In [19]:
def parse_GFN2(filename, molecule, data):
    '''
    Parses through GFN2-xTB output files

    Parameters
    -----------
    filename: str
        path to output file
    molecule: str
        name of molecule
    dictionary: list
        list of descriptors to add to
    '''
    
    with open(filename, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        while line:
            if 'molecular dipole' in line:
                line = file.readline()
                line = file.readline()
                line = file.readline()
                    
                line_list = line.split()
                dipole_moment = float(line_list[-1])
                
            elif 'Mol. C8AA' in line:
                line = file.readline()
                line_list = line.split()
                
                polarizability = float(line_list[-1])

            line = file.readline()  
        line = file.readline()

        outputs = [dipole_moment, polarizability]
        
        data.extend(outputs)

        return data
    
def parse_sTDA(filename, molecule, data):
    
    with open(filename, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        oscs = []
        wavelength = []
        energyEV = []
        while line:
            if 'ordered frontier orbitals' in line:
                for x in range(11):
                    line = file.readline()
                line_list = line.split()
                HOMOminus1 = float(line_list[1])
                
                line = file.readline()
                line_list = line.split()
                HOMO = float(line_list[1])
                
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                LUMO = float(line_list[1])
                line = file.readline()
                line_list = line.split()
                LUMOplus1 = float(line_list[1])

                deltaHOMO = abs(HOMOminus1 - HOMO)
                deltaLUMO = abs(LUMO - LUMOplus1)
                fundbg = abs(HOMO-LUMO)

            elif 'excitation energies, transition moments and TDA amplitudes' in line:
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                while line != '\n':
                    line_list = line.split()
                    oscs.append(float(line_list[3]))
                    wavelength.append(float(line_list[2]))
                    energyEV.append(float(line_list[1]))
                    line = file.readline()

            line = file.readline()  
        line = file.readline()
        
    chemical_potential = (HOMO + LUMO)/2
    hardness =  LUMO - HOMO
    # https://xtb-docs.readthedocs.io/en/latest/sp.html#global-electrophilicity-index
    electrophilicity = chemical_potential**2 / 2*hardness
   
    if len(oscs) != 0:
        summed_oscs = np.sum(oscs)
        
        highest_oscs = 0.0
        opt_bg = round(energyEV[0], 2)
        
        # Opt bg is the energy of the first transition within the first 12 transition with an oscillator strength greater than 0.5 
        if len(oscs) < 12:
            for i in range(len(oscs)):
                if  oscs[i] > 0.5:
                    opt_bg = round(energyEV[i], 2)
                    break
        else:
            for x in range(12):
                if  oscs[x] > 0.5:
                    opt_bg = round(energyEV[x], 2)
                    break

        # max abs is the tallest peak in the spectrum
        for x in range(len(oscs)):
            if  oscs[x] > highest_oscs:
                    highest_oscs = oscs[x]
                    max_abs = wavelength[x]
                    
        # Creates full spectrum
        (spectraEV, spectraNM, spectraIntensity) = spectra(energyEV, oscs)
        
        # Calculates the area under the curve using trapz rule for integration
        area_spectra = np.trapz(np.flip(spectraIntensity), np.flip(spectraNM), dx=0.1, axis=- 1)
        
        # Calculates the area under the curve of the simulated spectrum multiplied by the normalized solar spectrum
        area_sim_solar_spectra = solar_integrated_desc(spectraNM, spectraIntensity)
        
        outputs = [HOMO, HOMOminus1, LUMO, LUMOplus1, fundbg, deltaHOMO, deltaLUMO, opt_bg, max_abs, summed_oscs, area_spectra, area_sim_solar_spectra, chemical_potential, electrophilicity]

        data.extend(outputs)

        return data
    
    else:
        print(filename)
        print('something is wrong')
    

def spectra(etens, etoscs, low = 0.5, high = 10.0, resolution = 0.01, smear = 0.04):
    """
    Return arrays of the energies and intensities of a Lorentzian-blurred spectrum

    Parameters
    ----------
    etens: list
        list of transition energies in units of eV
    etoscs: list
        list of oscillator strengths
    low: float
        transition in eV to start spectrum at
    high: float
        transition in eV to end spectrum at
    resolution: float
        increments of eV for spectrum
    smear: float
        blurs intensities of peaks across 0.04 eV

    Returns
    -------
    Lists of the spectra in eV, nm, and their oscillator strengths
    """
    maxSlices = int((high - low) / resolution) + 1
    peaks = len(etens)

    spectraEV = []
    spectraNM = []
    spectraIntensity = []
    for i in range(0, maxSlices):
        energy = float(i * resolution + low) # units of eV
        #wavelength = energy * 1239.84193 # convert eV to nm  
        wavelength = 1239.84193 / energy # convert eV to nm  
        intensity = 0.0

        for trans in range(0, peaks):
            this_smear = smear / 0.2 * (-0.046 * etoscs[trans] + 0.20)
            deltaE = etens[trans] - energy
            intensity = intensity + etoscs[trans] * this_smear**2 / (deltaE**2 + this_smear**2)

        spectraEV.append(energy)
        spectraNM.append(wavelength) 
        spectraIntensity.append(intensity)
        
    return spectraEV, spectraNM, spectraIntensity

def custom_round(x, base=5):
    return float(base * round(float(x)/base))

def solar_integrated_desc(spectraNM, spectraIntensity):
    
    wavelength15AM = [] #wavelength for 1.5 AM spectra
    normalized_irr_15AM = []
    
    solar = pd.read_csv('../Solar_radiation_spectrum.csv', index_col = 'wavelength')
    new_spectrum_intensities = []
    
    # the 1.5AM solar spectra does not have constant increments of wavelengths
    for x in range(len(spectraNM)):
        
        if 280 <= spectraNM[x] < 400:
            int_wavelength = custom_round(spectraNM[x], 0.5)
        if 400 <= spectraNM[x] < 1700:
            int_wavelength = custom_round(spectraNM[x], 1)
        if 1700 <= spectraNM[x] < 1702:
            int_wavelength = custom_round(spectraNM[x], 2)
        if 1702 <= spectraNM[x] <=4000:
            int_wavelength = custom_round(spectraNM[x], 5)

        solar_intensity = solar.loc[int_wavelength][-1]
        
        new_spectrum_intensities.append(float(solar_intensity) * spectraIntensity[x])
        
    area_altered_spectra = np.trapz(np.flip(new_spectrum_intensities), np.flip(spectraNM), dx=0.1, axis=- 1)
    
    return area_altered_spectra


def getPiSystemSize(mol):
    mol = AllChem.RemoveHs(mol)
    AllChem.Kekulize(mol)
    pi_systems = [pi_system(mol,x.GetIdx(),[x.GetIdx()]) for x in mol.GetAtoms()]
    largest_pi_system = max(pi_systems, key=lambda coll: len(coll))
    pi_system_size = len(largest_pi_system)
    return pi_system_size

def pi_system(mol, current, seen):
    atom = mol.GetAtomWithIdx(current)
    for neighbor in atom.GetNeighbors():
        if (neighbor.GetIdx() not in seen) and (mol.GetBondBetweenAtoms(atom.GetIdx(),neighbor.GetIdx()).GetIsConjugated() or mol.GetBondBetweenAtoms(atom.GetIdx(),neighbor.GetIdx()).GetBondTypeAsDouble() > 1):
            seen.append(neighbor.GetIdx())
            pi_system(mol,neighbor.GetIdx(),seen)
    return seen

def pi_sys_size(filename, molecule, data):
    mol = AllChem.MolFromMolFile(filename)
    pi_size = getPiSystemSize(mol)
    
    outputs = [pi_size]

    data.extend(outputs)

    return data

def GetBestFitPlane(pts, weights=None):
    # number of atoms
    wSum = len(pts)
    # sets the origin as the sum of the coordinates for x, y, and z
    origin = np.sum(pts, 0)
    # finds the average of each coordinate and sets as the origin
    origin /= wSum

    # initiates blank coordinates
    sumXX = 0
    sumXY = 0
    sumXZ = 0
    sumYY = 0
    sumYZ = 0
    sumZZ = 0
    sums = np.zeros((3, 3), np.double)
    
    # finds the distance of each point to origin
    for pt in pts:
        # finds the distance of each point to origin
        dp = pt - origin
        
        # sets the 3x3 matrix
        for i in range(3):
            sums[i, i] += dp[i] * dp[i]
            for j in range(i + 1, 3):
                sums[i, j] += dp[i] * dp[j]
                sums[j, i] += dp[i] * dp[j]
    # Averages each number in matrix by the total number of atoms
    sums /= wSum
    
    # Finds the eigenvalues and eigenvectors 
    vals, vects = linalg.eigh(sums)

    # gives indices sorted from smallest to largest
    order = np.argsort(vals)
    
    # smallest eigenvector
    normal = vects[:, order[0]]    
    
    # sets plane coordinates
    plane = np.zeros((4, ), np.double)
    plane[:3] = normal
    plane[3] = -1 * normal.dot(origin)
    
    return plane


def PBFRD(mol, largest_pi_system, confId=-1):
    conf = mol.GetConformer(confId)
    if not conf.Is3D():
        return 0
    
    pts = np.array([list(conf.GetAtomPosition(x)) for x in largest_pi_system])
    plane = GetBestFitPlane(pts)
    
    #distance to point
    denom = np.dot(plane[:3], plane[:3])
    denom = denom**0.5
    # add up the distance from the plane for each point:
    res = 0.0
    for pt in pts:
        res += np.abs(pt.dot(plane[:3]) + plane[3])
        
    res /= denom
    res /= len(pts)
    
    # higher the number, the less planar it is
    return res

def planarity (filename, data):
    mol = Chem.MolFromMolFile(filename)
    mol = Chem.RemoveHs(mol)
    Chem.Kekulize(mol)
    pi_systems = [pi_system(mol,x.GetIdx(),[x.GetIdx()]) for x in mol.GetAtoms()]
    largest_pi_system = max(pi_systems, key=lambda coll: len(coll))

    planarity = PBFRD(mol, largest_pi_system)
    
    outputs = [planarity]
    data.extend(outputs)
    
    return data

def rdkit_descriptors(filename, data):
    mol = Chem.MolFromMolFile(filename)
    num_rot_bonds = Descriptors.NumRotatableBonds(mol)
    MolLogP = Descriptors.MolLogP(mol)
    TPSA = Descriptors.TPSA(mol)
    NumHAcceptors = Descriptors.NumHAcceptors(mol)
    NumHDonors = Descriptors.NumHDonors(mol)
    RingCount = Descriptors.RingCount(mol)

    outputs = [num_rot_bonds, MolLogP, TPSA, NumHAcceptors, NumHDonors, RingCount]
    
    data.extend(outputs)
    return data
    
def numpy_2_fp(array):
    fp = DataStructs.cDataStructs.UIntSparseIntVect(len(array))
    for ix, value in enumerate(array):
        fp[ix] = int(value)
    return fp
def morgan_fp_counts(filename, data):
    mol = Chem.MolFromMolFile(filename)
    fp3 = AllChem.GetHashedMorganFingerprint(mol, 2, nBits=2048)
    array = np.zeros((0,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp3, array)
    
    fp4 = numpy_2_fp(array)
    
    outputs = list(fp4)
    
    data.extend(outputs)
    return data

def solvation(filename, data):
    with open(filename, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        while line:
            if '-> Gsolv' in line:
                line_list = line.split()
                solvation_energy = float(line_list[3])
                break
                
            line = file.readline()  
        line = file.readline()

    outputs = [solvation_energy]
        
    data.extend(outputs)

    return data

In [8]:
acceptors = pd.read_csv('../top_200_acceptor_names.csv')
acceptors

,gen,filename,PCE,donor,GA
0,24,39_28_86_28_39,20.797438,PBCT-2F,2
1,24,39_50_86_28_39,20.493611,PBCT-2F,2
2,37,39_3_86_28_39,20.484320,PBCT-2F,2
3,21,39_28_86_15_39,20.454294,PBCT-2F,2
4,23,39_15_86_28_39,20.452430,PBCT-2F,2
...,...,...,...,...,...
195,63,39_15_86_157_39,19.425612,PBCT-2F,2
196,27,39_62_86_50_39,19.424567,PBCT-2F,2
197,21,39_115_39,19.422893,PBCT-2F,1
198,58,39_76_86_153_39,19.420608,PBCT-2F,2


In [20]:
column_names = ['acceptor', 'A-HOMO', 'A-HOMOminus1', 'A-LUMO', 'A-LUMOplus1', 'A-fundbg', 'A-deltaHOMO', 'A-deltaLUMO', 'A-opt_bg', 'A-max_abs', 'A-summed_oscs', 'A-area_spectra', 'A-area_sim_solar_spectra', 'A-chemical_potential', 'A-electrophilicity', 'A-pi_sys_size', 'A-num_rot_bonds', 'A-MolLogP', 'A-TPSA', 'A-NumHAcceptors', 'A-NumHDonors', 'A-RingCount','A-planarity','A-dipole_moment', 'A-polarizability', 'A-SolvationEnergy_water', 'A-SolvationEnergy_hexane']

# add column names for 2048 bit morgan fingerprints
for x in range(2048):
    col_name = 'A-ECFP_' + str(x)
    column_names.append(col_name)

df = pd.DataFrame(columns = column_names)

acc_seen = []
for x in range(len(acceptors)):

        acc = acceptors.iloc[x][1]
        
        if acc not in acc_seen:
            
            acc_seen.append(acc)
            data = []
            data.append(acc)
            
            GA_version = int(acceptors.iloc[x][4])
            
            if GA_version == 1:
                acc_stda = '../GA_1/rf_ann_fixed/sTDDFTxTB_output/' + acc + '.stda'
                acc_mol = '../GA_1/rf_ann_fixed/GFN2_output/' + acc + '.mol'
                acc_GFN2 = '../GA_1/rf_ann_fixed/GFN2_output/' + acc + '.out'
                acc_solv_water = '../GA_1/rf_ann_fixed/solvation_water/' + acc + '.out'
                acc_solv_hexane = '../GA_1/rf_ann_fixed/solvation_hexane/' + acc + '.out'
            elif GA_version == 2:
                acc_stda = '../GA_2/sTDDFTxTB_output/' + acc + '.stda'
                acc_mol = '../GA_2/GFN2_output/' + acc + '.mol'
                acc_GFN2 = '../GA_2/GFN2_output/' + acc + '.out'
                acc_solv_water = '../GA_2/solvation_water/' + acc + '.out'
                acc_solv_hexane = '../GA_2/solvation_hexane/' + acc + '.out'
            elif GA_version == 3:
                acc_stda = '../GA_3/sTDDFTxTB_output/' + acc + '.stda'
                acc_mol = '../GA_3/GFN2_output/' + acc + '.mol'
                acc_GFN2 = '../GA_3/GFN2_output/' + acc + '.out'
                acc_solv_water = '../GA_3/solvation_water/' + acc + '.out'
                acc_solv_hexane = '../GA_3/solvation_hexane/' + acc + '.out'
            else:
                print('bad GA version')
            

            # parse sTDDFT-xtb output files of donors
            parse_sTDA(acc_stda, acc, data) #HOMO, HOMOminus1, LUMO, LUMOplus1, fundbg, deltaHOMO, deltaLUMO, opt_bg, max_abs, summed_oscs, area_spectra, area_sim_solar_spectra, chemical_potential, electrophilicity

            # parse GFN2-xtb files for donor
            pi_sys_size(acc_mol, acc, data) #pi_size
            rdkit_descriptors(acc_mol, data) #num_rot_bonds, MolLogP, TPSA, NumHAcceptors, NumHDonors
            planarity(acc_mol, data) # planarity

            # calculate pi system size of donor
            parse_GFN2(acc_GFN2, acc, data) #dipole_moment, polarizability

            # calculate solvation free energy of donor in water
            solvation(acc_solv_water, data) #solvation_energy

            # calculate solvation free energy of donor in hexane
            solvation(acc_solv_hexane, data) #solvation_energy

            morgan_fp_counts(acc_mol, data)


            df.loc[len(df.index)] = data

df

,acceptor,A-HOMO,A-HOMOminus1,A-LUMO,A-LUMOplus1,A-fundbg,A-deltaHOMO,A-deltaLUMO,A-opt_bg,A-max_abs,...,A-ECFP_2038,A-ECFP_2039,A-ECFP_2040,A-ECFP_2041,A-ECFP_2042,A-ECFP_2043,A-ECFP_2044,A-ECFP_2045,A-ECFP_2046,A-ECFP_2047
0,39_28_86_28_39,-12.879,-13.161,-8.498,-8.478,4.381,0.282,0.020,2.46,390.0,...,0,0,0,0,0,0,0,0,0,0
1,39_50_86_28_39,-12.969,-13.419,-8.796,-8.454,4.173,0.450,0.342,2.37,522.9,...,0,0,0,0,0,0,0,0,0,0
2,39_3_86_28_39,-12.984,-13.300,-8.865,-8.498,4.119,0.316,0.367,2.54,407.9,...,0,0,0,0,0,0,0,0,0,0
3,39_28_86_15_39,-12.964,-13.271,-9.017,-8.641,3.947,0.307,0.376,2.59,421.1,...,0,0,0,0,0,0,0,0,0,0
4,39_15_86_28_39,-13.004,-13.324,-9.121,-8.762,3.883,0.320,0.359,2.89,350.7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,39_15_86_157_39,-13.220,-13.826,-9.035,-8.735,4.185,0.606,0.300,2.33,420.8,...,0,0,0,0,0,0,0,0,0,0
196,39_62_86_50_39,-13.232,-13.682,-8.971,-8.637,4.261,0.450,0.334,2.44,394.2,...,0,0,0,0,0,0,0,0,0,0
197,39_115_39,-13.579,-13.970,-8.858,-8.745,4.721,0.391,0.113,2.40,516.5,...,0,0,0,0,0,0,0,0,0,0
198,39_76_86_153_39,-13.229,-13.850,-9.061,-9.046,4.168,0.621,0.015,2.23,555.9,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df.to_csv('acc_descriptors_GA123_top200.csv')

# Statistics of acceptors

In [28]:
df.describe()

,A-HOMO,A-HOMOminus1,A-LUMO,A-LUMOplus1,A-fundbg,A-deltaHOMO,A-deltaLUMO,A-opt_bg,A-max_abs,A-summed_oscs,...,A-area_sim_solar_spectra,A-chemical_potential,A-electrophilicity,A-MolLogP,A-TPSA,A-planarity,A-dipole_moment,A-polarizability,A-SolvationEnergy_water,A-SolvationEnergy_hexane
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,...,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,-13.137540,-13.567655,-8.910655,-8.678455,4.226885,0.430115,0.232200,2.528400,454.206000,11.265669,...,54.874885,-11.024098,256.961858,31.017872,185.011350,1.171049,6.970060,1367.988903,-0.070425,-0.146033
std,0.181648,0.263920,0.135839,0.174545,0.199714,0.146339,0.151663,0.186228,48.352645,1.770850,...,15.563319,0.125509,15.110880,6.932909,22.325959,0.401196,2.625177,241.664909,0.007462,0.019476
min,-13.782000,-14.185000,-9.206000,-9.046000,3.805000,0.125000,0.015000,1.990000,282.900000,7.245100,...,16.701649,-11.344500,220.610886,13.425020,147.190000,0.330035,1.138000,747.609315,-0.095081,-0.183759
25%,-13.236000,-13.823750,-9.013000,-8.834750,4.129500,0.303500,0.086750,2.410000,418.850000,10.347150,...,44.551167,-11.112250,248.509474,25.348175,169.870000,0.877901,5.166250,1183.375247,-0.074939,-0.158230
50%,-13.106000,-13.460000,-8.935000,-8.637500,4.217000,0.433500,0.218500,2.520000,455.300000,11.067700,...,55.187048,-11.037000,255.128382,32.375150,179.275000,1.167476,6.971000,1400.967326,-0.070094,-0.148913
75%,-13.024000,-13.354750,-8.842250,-8.517750,4.307500,0.565250,0.367500,2.632500,490.600000,12.325775,...,64.134305,-10.950250,263.011734,35.037130,197.705000,1.480341,8.834750,1523.929907,-0.065723,-0.134543
max,-12.650000,-12.926000,-8.457000,-8.411000,4.898000,0.691000,0.638000,3.230000,595.400000,16.565500,...,127.489225,-10.659000,313.700596,45.265520,291.260000,2.057676,15.082000,1890.393493,-0.053981,-0.089016


In [24]:
def acc_to_csv(filename, acc):
    with open(filename, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        oscs = []
        energyEV = []
        while line:
            if 'excitation energies, transition moments and TDA amplitudes' in line:
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                while line != '\n':
                    line_list = line.split()
                    oscs.append(float(line_list[3]))
                    energyEV.append(float(line_list[1]))
                    line = file.readline()
            line = file.readline()  
        line = file.readline()


        # Creates full spectrum
        (acc_spectraEV,  acc_spectraNM, acc_spectraIntensity) = spectra(energyEV, oscs)

        df_acc = pd.DataFrame()
        df_acc['eV'] = acc_spectraEV
        df_acc['nm'] = acc_spectraNM
        df_acc['intensity'] = acc_spectraIntensity

        filename_abs = 'acc_abs_csvs/' + acc + '_abs.csv'
        df_acc.to_csv(filename_abs)
        
for x in range(len(acceptors)):

    acc = acceptors.iloc[x][1]
    GA_version = int(acceptors.iloc[x][4])

    if GA_version == 1:
        acc_stda = '../GA_1/rf_ann_fixed/sTDDFTxTB_output/' + acc + '.stda'

    elif GA_version == 2:
        acc_stda = '../GA_2/sTDDFTxTB_output/' + acc + '.stda'

    elif GA_version == 3:
        acc_stda = '../GA_3/sTDDFTxTB_output/' + acc + '.stda'

    else:
        print('bad GA version')

    acc_to_csv(acc_stda, acc)